In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
X,y = make_regression(n_samples=1000, n_features=3, n_targets=1, noise=.2)

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_train = torch.from_numpy(X_train).type(torch.float32)
y_train = torch.from_numpy(y_train).type(torch.float32)
X_test = torch.from_numpy(X_test).type(torch.float32)
y_test = torch.from_numpy(y_test).type(torch.float32)
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 3) (800,)
(200, 3) (200,)
torch.Size([800, 3]) torch.Size([800, 1])
torch.Size([200, 3]) torch.Size([200, 1])


 - to create dataset you have to implement below methods
     - __getitem__
     - **__len__**
     
     
 - to get the info from object yiu can implement below
     - __str__

In [3]:
## Create dataset
class CustomDataset(Dataset):
    
    def __init__(self, X,y):
        self.X = X
        self.y = y
        self.samples = X.shape[0]
    
    ## mandatory
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    ## mandatory
    def __len__(self):
        return self.samples
    
    
    ## optional
    def __str__(self):
        return f"Dataset having samples : {self.samples}"

In [4]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [5]:
first_sample = train_dataset[0]
features, labels = first_sample
print(features)
print(labels)

tensor([-0.6658,  0.1123, -1.3694])
tensor([-183.1725])


In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)

In [7]:
examples = iter(train_dataloader)
samples = examples.next()
features, labels = samples
print(features)
print(labels)

tensor([[ 0.1037, -0.6018,  1.0321],
        [-0.5294,  0.3544,  0.7822],
        [ 0.7193,  0.0928,  0.8918],
        [ 1.5350, -0.9800, -1.3872]])
tensor([[ 82.6825],
        [ 43.4477],
        [150.7231],
        [-39.9409]])


In [8]:
## Linear Model
class RegressionNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
sampels, features = X_train.shape
input_size = features
hidden_size = 5
output_size = 1
lr = 0.001
n_iters = 15

model = RegressionNet(input_size, hidden_size, output_size)
criteria = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

## Training loop
for epoch in range(n_iters):
    for i,(inputs, targets) in enumerate(train_dataloader):
        y_preds = model(inputs)
        loss = criteria(y_preds,targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    if (epoch+1)%2==0:
        print(f"Epoch : {epoch}, loss : {loss.item():.4f}")

Epoch : 1, loss : 0.1933
Epoch : 3, loss : 0.0799
Epoch : 5, loss : 0.0176
Epoch : 7, loss : 0.1236
Epoch : 9, loss : 0.1094
Epoch : 11, loss : 0.0191
Epoch : 13, loss : 0.0213


In [10]:
## Validation
running_loss = 0.0
for i,(inputs, targets) in enumerate(test_dataloader):
        y_preds = model(inputs)
        loss = criteria(y_preds,targets)
        running_loss += loss.item()
print(f"Loss : {running_loss/(i+1)}")

Loss : 0.03976575092878193
